In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv(r"C:\Users\lenwe\Downloads\diabetes.csv")
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [6]:
df['Glucose'] = np.where(df.Glucose == 0, df.Glucose.median(), df['Glucose'])
df['Insulin'] = np.where(df.Insulin == 0, df.Insulin.median(), df['Insulin'])

In [7]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

In [9]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=10).fit(X_train, y_train)
prediction = rf.predict(X_test)

In [10]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

confusion_matrix(y_test, prediction)

array([[86, 13],
       [25, 30]], dtype=int64)

In [15]:
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

           0       0.77      0.87      0.82        99
           1       0.70      0.55      0.61        55

    accuracy                           0.75       154
   macro avg       0.74      0.71      0.72       154
weighted avg       0.75      0.75      0.75       154



In [12]:
accuracy_score(y_test, prediction)

0.7532467532467533

In [16]:
model = RandomForestClassifier(n_estimators=500, criterion='gini',
                               max_features='sqrt', min_samples_leaf=10, random_state=100).fit(X_train, y_train)
prediction = model.predict(X_test)

In [17]:
confusion_matrix(y_test, prediction)

array([[87, 12],
       [22, 33]], dtype=int64)

In [18]:
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

           0       0.80      0.88      0.84        99
           1       0.73      0.60      0.66        55

    accuracy                           0.78       154
   macro avg       0.77      0.74      0.75       154
weighted avg       0.78      0.78      0.77       154



In [19]:
accuracy_score(y_test, prediction)

0.7792207792207793

# Random Search

In [21]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start=200, stop=2000, num=10)]

# Number of featuers to consider at every split
max_features = ['auto', 'sqrt', 'log2']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 1000, 10)]

# Minimum number of samples required to split a node
min_samples_split = [1, 2, 3, 4, 5, 7, 9]

# Minimum number of samples required at each leaf node
min_samples_leaf =  [1, 2, 4, 6, 8]

# Create thre random grid
random_grid = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'criterion': ['entropy', 'gini']
}

print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [1, 2, 3, 4, 5, 7, 9], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


In [27]:
rf = RandomForestClassifier()
rf_randomcv = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,
                                n_iter=100, cv=3, verbose=2, random_state=100, n_jobs=-1)

In [28]:
rf_randomcv.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


C:\Users\lenwe\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
48 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
48 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\lenwe\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\lenwe\AppData\Roaming\Python\Python38\site-packages\sklearn\ensemble\_forest.py", line 340, in fit
    self._validate_params()
  File "C:\Users\lenwe\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py", line 600, in _validate_params
    validate

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'criterion': ['entropy', 'gini'],
                                        'max_depth': [10, 120, 230, 340, 450,
                                                      560, 670, 780, 890,
                                                      1000],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4, 6, 8],
                                        'min_samples_split': [1, 2, 3, 4, 5, 7,
                                                              9],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=100, verbose=2)

In [29]:
rf_randomcv.best_params_

{'n_estimators': 400,
 'min_samples_split': 4,
 'min_samples_leaf': 8,
 'max_features': 'sqrt',
 'max_depth': 450,
 'criterion': 'entropy'}

In [30]:
randomcv_best_params = rf_randomcv.best_estimator_

In [31]:
y_pred = randomcv_best_params.predict(X_test)

In [32]:
confusion_matrix(y_test, y_pred)

array([[85, 14],
       [20, 35]], dtype=int64)

In [33]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.88      0.84        99
           1       0.73      0.60      0.66        55

    accuracy                           0.78       154
   macro avg       0.77      0.74      0.75       154
weighted avg       0.78      0.78      0.77       154



In [34]:
accuracy_score(y_test, y_pred)

0.7792207792207793

# Grid Search

In [35]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'criterion': [rf_randomcv.best_params_['criterion']],
    'max_depth': [rf_randomcv.best_params_['max_depth']],
    'max_features': [rf_randomcv.best_params_['max_features']],
    'min_samples_leaf': [rf_randomcv.best_params_['min_samples_leaf'],
                         rf_randomcv.best_params_['min_samples_leaf'] + 2,
                         rf_randomcv.best_params_['min_samples_leaf'] + 4],
    'min_samples_split': [rf_randomcv.best_params_['min_samples_split'] - 2,
                          rf_randomcv.best_params_['min_samples_split'] - 1,
                          rf_randomcv.best_params_['min_samples_split'],
                          rf_randomcv.best_params_['min_samples_split'] + 1,
                          rf_randomcv.best_params_['min_samples_leaf'] + 2],
    'n_estimators': [rf_randomcv.best_params_['n_estimators'] - 200,
                    rf_randomcv.best_params_['n_estimators'] - 100,
                    rf_randomcv.best_params_['n_estimators'],
                    rf_randomcv.best_params_['n_estimators'] + 100,
                    rf_randomcv.best_params_['n_estimators'] + 200,
                    rf_randomcv.best_params_['n_estimators'] - 600,]
}

print(param_grid)

{'criterion': ['entropy'], 'max_depth': [450], 'max_features': ['sqrt'], 'min_samples_leaf': [8, 10, 12], 'min_samples_split': [2, 3, 4, 5, 10], 'n_estimators': [200, 300, 400, 500, 600, -200]}


In [36]:
rf =  RandomForestClassifier()
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=10, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 90 candidates, totalling 900 fits


C:\Users\lenwe\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
150 fits failed out of a total of 900.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
150 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\lenwe\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\lenwe\AppData\Roaming\Python\Python38\site-packages\sklearn\ensemble\_forest.py", line 340, in fit
    self._validate_params()
  File "C:\Users\lenwe\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py", line 600, in _validate_params
    valida

GridSearchCV(cv=10, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['entropy'], 'max_depth': [450],
                         'max_features': ['sqrt'],
                         'min_samples_leaf': [8, 10, 12],
                         'min_samples_split': [2, 3, 4, 5, 10],
                         'n_estimators': [200, 300, 400, 500, 600, -200]},
             verbose=2)

In [37]:
grid_search.best_estimator_

RandomForestClassifier(criterion='entropy', max_depth=450, min_samples_leaf=8,
                       min_samples_split=10, n_estimators=200)

In [38]:
best_grid = grid_search.best_estimator_

In [39]:
y_pred = best_grid.predict(X_test)

In [40]:
confusion_matrix(y_test, y_pred)

array([[87, 12],
       [20, 35]], dtype=int64)

In [44]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.88      0.84        99
           1       0.74      0.64      0.69        55

    accuracy                           0.79       154
   macro avg       0.78      0.76      0.77       154
weighted avg       0.79      0.79      0.79       154



In [43]:
accuracy_score(y_test, y_pred)

0.7922077922077922

# Bayesian Optimization

In [47]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

In [48]:
space = {
    'criterion': hp.choice('criterion', ['entropy', 'gini']),
    'max_depth': hp.quniform('max_depth', 10, 1200, 10),
    'max_featuers': hp.choice('max_features', ['auto', 'sqrt', 'log2', None]),
    'min_samples_leaf': hp.uniform('min_samples_leaf', 0, 0.5),
    'min_samples_split': hp.uniform('min_samples_split', 0, 1),
    'n_estimators': hp.choice('n_estimators', [10, 50, 300, 750, 1200, 1300, 1500])
}

In [51]:
def objective(space):
    model = RandomForestClassifier(criterion=space['criterion'], max_depth=int(space['max_depth']),
                                  max_features=space['max_featuers'], min_samples_leaf=space['min_samples_leaf'],
                                  n_estimators=space['n_estimators'])

    accuracy = cross_val_score(model, X_train, y_train, cv=5).mean()
    
    # We aim to maximize accuracy, therefore we return it as a negative value
    return {'loss': -accuracy, 'status': STATUS_OK}

In [52]:
from sklearn.model_selection import cross_val_score
trials = Trials() # it is responsible for minimizing the function
best = fmin(fn=objective,           
           space=space, 
           algo=tpe.suggest, 
           max_evals=80, 
           trials=trials)
best

  5%|▌         | 4/80 [00:15<04:52,  3.85s/trial, best loss: -0.7573104091696654]

C:\Users\lenwe\AppData\Roaming\Python\Python38\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(

C:\Users\lenwe\AppData\Roaming\Python\Python38\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(

C:\Users\lenwe\AppData\Roaming\Python\Python38\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or re

 10%|█         | 8/80 [00:30<04:35,  3.83s/trial, best loss: -0.7573104091696654]

C:\Users\lenwe\AppData\Roaming\Python\Python38\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(

C:\Users\lenwe\AppData\Roaming\Python\Python38\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(

C:\Users\lenwe\AppData\Roaming\Python\Python38\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or re

 14%|█▍        | 11/80 [00:42<04:27,  3.88s/trial, best loss: -0.7573104091696654]

...

C:\Users\lenwe\AppData\Roaming\Python\Python38\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(

C:\Users\lenwe\AppData\Roaming\Python\Python38\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(

C:\Users\lenwe\AppData\Roaming\Python\Python38\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or re

 92%|█████████▎| 74/80 [03:49<00:02,  2.10trial/s, best loss: -0.7703585232573638]

C:\Users\lenwe\AppData\Roaming\Python\Python38\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(

C:\Users\lenwe\AppData\Roaming\Python\Python38\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(



100%|██████████| 80/80 [03:58<00:00,  2.98s/trial, best loss: -0.7703585232573638]


{'criterion': 1,
 'max_depth': 20.0,
 'max_features': 1,
 'min_samples_leaf': 0.07418229201934401,
 'min_samples_split': 0.7109749568267631,
 'n_estimators': 0}

In [53]:
crit = {0: 'entropy', 1: 'gini'}
feat = {0:'auto', 1: 'sqrt', 2: 'log2', 3: None}
est = {0:10, 1: 50, 2:300, 4: 1200, 5:1300, 6: 1500}

print(crit[best['criterion']])
print(feat[best['max_features']])
print(est[best['n_estimators']])

gini
sqrt
10


In [55]:
trainedforest = RandomForestClassifier(criterion=crit[best['criterion']], max_depth=int(best['max_depth']),
                                      max_features=feat[best['max_features']], min_samples_leaf=best['min_samples_leaf'],
                                      min_samples_split=best['min_samples_split'], n_estimators=est[best['n_estimators']]).fit(X_train, y_train)

predictionforest = trainedforest.predict(X_test)

In [56]:
confusion_matrix(y_test, predictionforest)

array([[99,  0],
       [55,  0]], dtype=int64)

In [58]:
print(classification_report(y_test, predictionforest))

              precision    recall  f1-score   support

           0       0.64      1.00      0.78        99
           1       0.00      0.00      0.00        55

    accuracy                           0.64       154
   macro avg       0.32      0.50      0.39       154
weighted avg       0.41      0.64      0.50       154



C:\Users\lenwe\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lenwe\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lenwe\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [59]:
accuracy_score(y_test, predictionforest)

0.6428571428571429